In [20]:
%load_ext autoreload
%autoreload 2

from helpers import *

import bz2
# import json

import pandas as pd
import numpy as np
# import pickle

import sklearn as sk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import accuracy_score

from scipy.sparse import vstack



#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
import gensim
from gensim.models import Word2Vec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giomonopoli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/giomonopoli/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/giomonopoli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
path_to_file = 'data.nosync/processed/quotes-2020-party_labeled_cleaned.json.bz2'

In [22]:
df = pd.read_json(path_to_file, lines=True, compression='bz2')

In [23]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

In [24]:
def change_lb(x):
    if x == 'R':
        return 1
    else:
        return 0

df["party_label"] = df["party_label"].apply(lambda x: change_lb(x))
y = df["party_label"]

In [6]:
df['clean_quotation'] = df['quotation'].apply(lambda x: preprocess(x))
df = df.drop_duplicates(subset=['clean_quotation'])

In [7]:
#correct proportion (doesnt change nothing)
rows = np.random.choice(df[df["party_label"] == 0].index.values, 100000)
sampled_df = df.loc[rows]
rows = np.random.choice(df[df["party_label"] == 1].index.values, 100000)
sampled2_df = df.loc[rows]
balanced_df = pd.concat([sampled_df,sampled2_df])

In [8]:
train_size=195000
X_mini = balanced_df['clean_quotation'][:train_size]
X_test = balanced_df['clean_quotation'][train_size:train_size + 5000]
y_mini = y[:train_size]
y_test = y[train_size:train_size + 5000]

In [9]:
def simple_tokeniser(text):
    return text.split()

with open("stopwords.txt") as f:
    stop_words = list(map(lambda x: x[:-1], f.readlines()))
vectorizer = TfidfVectorizer(stop_words=stop_words,tokenizer=simple_tokeniser, min_df=2)
train_vectors =vectorizer.fit_transform(X_mini)
test_vectors = vectorizer.transform(X_test)

FileNotFoundError: [Errno 2] No such file or directory: 'stopwords.txt'

In [88]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(train_vectors, y_mini)  
#Predict y value for test dataset
y_predict = nb_tfidf.predict(test_vectors)
y_prob = nb_tfidf.predict_proba(test_vectors)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

NameError: name 'train_vectors' is not defined

In [89]:
#il numero di republicani vs democratici non dev esser proprozionale? need to balance out

## TEST

#### Preprocessing

In [192]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
nltk_stop_words = stopwords.words('english')
tf_vectorizer = CountVectorizer(max_df=0.8, min_df=50,
    ngram_range = (1,2),
    binary=True,
    stop_words=nltk_stop_words)

In [245]:
term_frequencies = tf_vectorizer.fit_transform(df['quotation_cleanD'].values)
phrases_df = pd.DataFrame(data=tf_vectorizer.get_feature_names_out(),columns=['phrase'])
phrases_df['total_occurrences']=np.array(term_frequencies.sum(axis=0)).reshape(term_frequencies.sum(axis=0).shape[1])
phrases_df.sort_values(by=['total_occurrences','phrase'],ascending=False).head(300).to_csv('top_20_overall.csv',index=False)

In [246]:
"""from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(categories=['talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',])"""

"from sklearn.datasets import fetch_20newsgroups\nnewsgroups_train = fetch_20newsgroups(categories=['talk.politics.guns',\n 'talk.politics.mideast',\n 'talk.politics.misc',])"

In [25]:
path = fixpath(QUOTES_ALL_TIME_PROCESSED)

df = pd.read_json(path, orient='records', lines=True)
def downsample(df:pd.DataFrame, label_col_name:str) -> pd.DataFrame:
    # find the number of observations in the smallest group
    nmin = df[label_col_name].value_counts().min()
    return (df
            # split the dataframe per group
            .groupby(label_col_name)
            # sample nmin observations from each group
            .apply(lambda x: x.sample(nmin))<§§
            # recombine the dataframes
            .reset_index(drop=True)
            )

df = downsample(df, 'party_label')
#X = df['quotation_cleanD'].values
#print(len(X))
#y = df['party_label'].values
#y = np.array([0 if label=='R' else 1 for label in y])

In [26]:
X = df['quotation_clean'].values
print(len(X))
y = df['party_label'].values
y = np.array([0 if label=='R' else 1 for label in y])

6343692


In [27]:
common_w = pd.read_csv("top_20_overall.csv")
common_w['phrase'].values

array(['peopl', 'go', 'state', 'need', 'get', 'think', 'make', 'work',
       'presid', 'time', 'one', 'want', 'would', 'know', 'like', 'us',
       'take', 'right', 'american', 'health', 'come', 'year', 'thing',
       'way', 'new', 'trump', 'help', 'say', 'see', 'continu', 'look',
       'communiti', 'day', 'support', 'public', 'countri', 'everi',
       'realli', 'care', 'back', 'protect', 'mani', 'hous', 'put',
       'nation', 'senat', 'well', 'good', 'could', 'lot', 'import',
       'famili', 'govern', 'busi', 'use', 'sure', 'tri', 'live',
       'democrat', 'vote', 'provid', 'even', 'issu', 'first', 'today',
       'got', 'also', 'believ', 'much', 'elect', 'keep', 'home', 'call',
       'bill', 'respons', 'place', 'abl', 'polit', 'job', 'person',
       'chang', 'done', 'make sure', 'decis', 'togeth', 'case', 'world',
       'someth', 'unit', 'happen', 'last', 'week', 'law', 'said', 'hope',
       'covid', 'coronavirus', 'test', 'talk', 'made', 'citi', 'long',
       'best', 'mu

In [28]:
def process(x):
    arr = x.split(" ")
    return any(x in common_w['phrase'].values for x in arr)


mask = df['quotation_clean'].apply(lambda x: process(x))
X= df['quotation_clean'][mask]
y = y[mask]

In [29]:
df

,quoteID,quotation,speaker,date,numOccurrences,id,party_label,US_congress_bio_ID,quotation_clean
0,2020-01-06-040043,I've already started working with my colleague...,eliot engel,2020-01-06 01:53:44,1,Q1329618,D,E000179,already started working colleagues new legisla...
1,2018-11-02-039720,I have known Islam on three continents before ...,barack obama,2018-11-02 04:06:00,1,Q76,D,O000167,known islam three continents coming region fir...
2,2018-02-06-101815,tax breaks for the wealthiest one percent,nancy pelosi,2018-02-06 09:00:06,1,Q170581,D,P000197,tax breaks wealthiest one percent
3,2018-04-22-083854,"When you go on a mission, you don't ask, `Who ...",mikie sherrill,2018-04-22 11:27:02,1,Q47087146,D,S001207,go mission ask democrat republican
4,2018-04-17-000547,"[ T ] oo often, consumers don't have the basic...",eric schneiderman,2018-04-17 15:20:39,5,Q5387459,D,None,oo often consumers basic facts need assess fai...
...,...,...,...,...,...,...,...,...,...
6343687,2017-12-22-070993,significant long-term effort to heal the polit...,heather nauert,2017-12-22 00:00:00,3,Q5694014,R,None,significant long term effort heal political di...
6343688,2016-02-16-051709,"I'm 44, I feel 46. I messed up the joke -- I'l...",marco rubio,2016-02-16 18:52:50,2,Q324546,R,R000595,feel messed joke may feel happens repeat somet...
6343689,2017-12-20-011661,become clear that Congress will only be able t...,susan collins,2017-12-20 18:30:32,2,Q22279,R,C001035,become clear congress able pass another short ...
6343690,2018-05-07-106756,Those people (against brothels) don't understa...,dennis hof,2018-05-07 02:00:00,1,Q5258518,R,None,people brothels understand nevada


#### Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=4)
#This works for individual job description, but how to save the model? 
tdidf = TfidfVectorizer(ngram_range=(1,3))
fitted_vectorizer = tdidf.fit(X_train)
tfidf_vectorizer_vectors = fitted_vectorizer.transform(X_train)

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
#nb_tfidf.fit(tfidf_vectorizer_vectors, y_train)


In [ ]:
scores = cross_val_score(nb_tfidf, tfidf_vectorizer_vectors, y_train, cv=5)

print(f'Avg: {scores.mean():.3f}\tStd: {scores.std():.3f}')

In [ ]:
#Predict y value for test dataset
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(tfidf_vectorizer_vectors, y_train)
y_predict = nb_tfidf.predict(fitted_vectorizer.transform(X_test))
y_prob = nb_tfidf.predict_proba(fitted_vectorizer.transform(X_test))[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)